## Notebook to visualize cell-type importance based on colocalziation between PD risk and cell-specific eQTL

modified off the colocalization heatmap notebook

In [ ]:
!date

#### import libraries

In [ ]:
from pandas import read_csv, concat, read_parquet, DataFrame
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
from seaborn import barplot, color_palette
from os.path import exists

%matplotlib inline
# for white background of figures (only for docs rendering)
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

#### set notebook variables

In [ ]:
# naming
cohort = 'foundin'
dx = 'PD'
day = 'daNA'

# directories
wrk_dir = '/labshare/raph/datasets/foundin_qtl'
results_dir = f'{wrk_dir}/results'
public_dir = f'{wrk_dir}/public'

# out files
index_variants_file = f'{public_dir}/nalls_pd_gwas/index_variants.list' 

# variables
DEBUG = False
modalities = ['Bryois-Astro', 'Bryois-Endo', 'Bryois-ExN', 'Bryois-InN', 
              'Bryois-Micro', 'Bryois-OPC', 'Bryois-Oligo', 'Bryois-Peri', 
              'DAn-meta']
min_h4 = 0.5

### for each day and modality load the colocalization results

In [ ]:
coloc_df = None
for modality in modalities:
    print(day, modality, end=':')
    in_file = f'{results_dir}/{cohort}_{day}_{modality}_{dx}.coloc.pp.csv'
    if exists(in_file):
        this_df = read_csv(in_file)
        print(f'loaded {this_df.shape[0]} results')
        # add day and modality
        this_df['day'] = day
        this_df['modality'] = modality
        coloc_df = concat([coloc_df, this_df])
print(f'\ntotal results loaded {coloc_df.shape[0]}')
if DEBUG:
    display(coloc_df.sample(5))
    display(coloc_df.day.value_counts())
    display(coloc_df.modality.value_counts()) 

### subset based on the minimum H4 variable

In [ ]:
temp = coloc_df.loc[coloc_df.H4 > min_h4]
print(f'{temp.feature.nunique()} traits will be used')
coloc_df = coloc_df.loc[coloc_df.feature.isin(temp.feature)]
print(f'results shape after filter on H4 {coloc_df.shape}')
if DEBUG:
    display(coloc_df.head())
    display(coloc_df.modality.value_counts()) 

### sum H4 probabilities by modality
since here just using single day = daNA don't have to worry about multiple days

In [ ]:
def sum_h4s(df: DataFrame, weighted: bool=False, verbose: bool=False) -> DataFrame:
    if weighted:
        cell_type_sums = df.groupby(by='modality').agg({'weighted_h4': 'sum'})
        append_str = 'weighted'
    else:
        cell_type_sums = df.groupby(by='modality').agg({'H4': 'sum'})
        append_str = ''
    print(f'shape of cell-type {append_str} H4 sums is {cell_type_sums.shape}')
    cell_type_sums['cell-type'] = cell_type_sums.index
    if DEBUG:
        display(cell_type_sums)
    return cell_type_sums

def plot_importance(sums_df: DataFrame, weighted: bool=False, dpi_value: int=100):
    if weighted:
        key = 'weighted_h4'
        key_label = 'weighted H4'
    else:
        key_label = key = 'H4'
    temp = sums_df.sort_values(key, ascending=False)
    with rc_context({'figure.figsize': (9, 9), 'figure.dpi': dpi_value}):
        plt.style.use('seaborn-v0_8-bright')    
        barplot(data=temp, x=key, y='cell-type', color='Purple')    
        plt.title('Cell-type importance based on risk and eQTL colocalization')
        plt.xlabel(f'Sum of {key_label}')
        plt.ylabel('Cell Type')
        plt.show()

In [ ]:
cell_type_sums = sum_h4s(coloc_df, verbose=DEBUG)

### visualize to cell importance based on cell-type H4 sum

In [ ]:
plot_importance(cell_type_sums)

### weight colocalizations by risk effect

#### load the risk variants of interest

In [ ]:
%%time
index_vars_df = read_csv(index_variants_file)
print(index_vars_df.shape)
index_variants = list(index_vars_df.variant.unique())
if DEBUG:
    display(index_vars_df.head())
    print(index_variants)

#### load variant colocalization info

In [ ]:
causals_df = None
for modality in modalities:
    print(day, modality, end=':')
    in_file = f'{results_dir}/{cohort}_{day}_{modality}_{dx}.casuals.pp.parquet'
    if exists(in_file):
        this_df = read_parquet(in_file)
        print(f'loaded {this_df.shape[0]} results')
        if 'MarkerName' in this_df.columns:
            this_df['feature'] = this_df.MarkerName.str.split(':', expand=True)[0]
        else:
            this_df = this_df.rename(columns={'phenotype_id': 'feature'})
        # add day and modality
        this_df['day'] = day
        this_df['modality'] = modality
        keep_cols = ['beta', 'logABF_risk', 'PP_risk',
                     'logABF_qtl',  'PP_qtl',
                     'h4_pp', 'feature', 'day', 'modality']        
        # subset to consistent cols prior to concat
        this_df = this_df[keep_cols]
        causals_df = concat([causals_df, this_df])
print(f'\ntotal results loaded {causals_df.shape[0]}')
# subset to just the results that intersect with risk indices
causals_df = causals_df.loc[causals_df.index.isin(index_variants)]
print(f'\risk index results loaded {causals_df.shape[0]}')
if DEBUG:
    display(causals_df.sample(5))
    display(causals_df.day.value_counts())
    display(causals_df.modality.value_counts()) 

#### merge the coloc H4 with the causals H4_PP

In [ ]:
causals_df['variant'] = causals_df.index
combine_df = coloc_df.merge(causals_df, how='inner', on=['feature', 'modality'], 
                            suffixes=['_coloc', '_causals'])
print(f'combined shape is {combine_df.shape}')
if DEBUG:
    display(combine_df.sample(5))

#### compute the weighted value

In [ ]:
combine_df['weighted_h4'] = combine_df.H4 * combine_df.beta.abs()
# combine_df['weighted_h4'] = combine_df.H4 * combine_df.logABF_risk
print(f'modified combined shape is {combine_df.shape}')
if DEBUG:
    display(combine_df.sample(5))

### visualize to cell importance based on cell-type H4 sum

In [ ]:
cell_type_sums = sum_h4s(combine_df, weighted=True, verbose=DEBUG)
plot_importance(cell_type_sums, weighted=True)

### assign feature to cell based on best H4 value

In [ ]:
coloc_best_df = coloc_df.sort_values(by='H4', ascending=False).drop_duplicates(['feature'], keep='first')
print(f'shape of coloc tops {coloc_best_df.shape}')
if DEBUG:
    display(coloc_best_df.sample(5))

In [ ]:
feature = 'CCAR2'
display(coloc_best_df.loc[coloc_best_df.feature == feature])
display(coloc_df.loc[coloc_df.feature == feature])

### visualize to cell importance based on cell-type H4 sum

In [ ]:
cell_type_sums = sum_h4s(coloc_best_df, verbose=DEBUG)
plot_importance(cell_type_sums)

In [ ]:
!date